In [1]:
"""
check
https://www.parlamento.it/1233  --> link per senatori e deputati
"""

'\ncheck\nhttps://www.parlamento.it/1233  --> link per senatori e deputati\n'

In [2]:
#!pip install html5lib
#!pip install bs4
#!pip install pdf2image
#!pip install fitz
# #!pip install PyMuPDF
# !pip install pytesseract
# !pip install pillow
#!sudo port install tesseract
#!pip install opencv-python

### imports
import pandas as pd
import requests
import pdf2image

In [3]:
import requests
from bs4 import BeautifulSoup
from PIL import Image
from pytesseract import pytesseract
import string
import pandas as pd
import fitz
import os
import tqdm
import cv2
import numpy as np

check structure of url from ex https://www.senato.it/leg/18/BGT/Schede/Attsen/Sena.html 
Final letter after 'Sen' is the alphabetical letter of family name

In [4]:
# want to iterate through all the letters of the alphabet
alphabet = list(string.ascii_lowercase)
print(alphabet)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [5]:
d = []

for i in alphabet:
    #url for each value of the alphabet
    URL = "https://www.senato.it/leg/18/BGT/Schede/Attsen/Sen{}.html".format(i)
    print(i, end='')
    try:
        r = requests.get(URL)
        soup = BeautifulSoup(r.content)
        mydivs = soup.find_all("div", {"class": "linkSenatore"})
        for x in mydivs:
            # 1st iteration:
            # for each name starting with 'A' I take the full name,
            # the party of affiliation and the senato code
            content = x.find_all('a', href=True)
            d.append(
                {
                    'full_name': content[0].text,
                    'affilitation': content[1].text,
                    'code': x.find_all('a') [0]['href'] .split("&id=",1)[1]
                }
            )        
    except:
        print('.')

abcdefghijklmnop.
qr.
stuvwxyz

In [6]:
# create df with relevant infos
df = pd.DataFrame(d)
df['family_name'] = (
    df['full_name'].str.split().str[0] # extract family name from full name,
    .str.title()  #first letter uppercase rest lower as for url of declaration
)
# next create url to declaration
url_declaration = 'http://senato.it/application/xmanager/projects/leg18/file/repository/documentaz_patr_17/2021/'
df['declaration_web'] = url_declaration + df['family_name'].astype(str) + '_2021.pdf?'

df.to_csv('df_base.csv', index=False)
                                               
df.head(2)


,full_name,affilitation,code,family_name,declaration_web
0,ABATE Rosa Silvana,UpC-CAL-Alt-PC-AI-Pr.SMART-IdV,32576,Abate,http://senato.it/application/xmanager/projects...
1,ACCOTO Rossella,M5S,32577,Accoto,http://senato.it/application/xmanager/projects...


check for example for the URL of declaration of one member of the senato
https://www.senato.it/application/xmanager/projects/leg18/file/repository/documentaz_patr_17/2021/Abate_2021.pdf

In [8]:
#next we create as many folder as for the row and we store in them the 'dichiarazione dei redditi'

In [9]:
folder_dichiarazione='dichiarazioni'

# create a folder to store dichiarazione dei redditi
current_directory = os.getcwd()
final_directory = os.path.join(current_directory, folder_dichiarazione)
if not os.path.exists(final_directory):
   os.makedirs(final_directory)

#for x in df['declaration_web'].to_list():


for a, b in tqdm.tqdm(df[['family_name','declaration_web']].itertuples(index=False), total=df.shape[0]):
    response = requests.get(b)
    f = open(
        os.path.join(folder_dichiarazione, a+'.pdf'),
        'wb'
    )
    f.write(response.content)

100%|█████████████████████████████████████████| 315/315 [09:28<00:00,  1.81s/it]


In [10]:
dpi = 300  # choose desired dpi here
zoom = dpi / 72  # zoom factor, standard: 72 dpi
magnify = fitz.Matrix(zoom, zoom)  # magnifies in x, resp. y direction

def split_doc_png(page):
    """ split image into single page png
    files"""
    pix = page.get_pixmap(matrix=magnify)  # render page to an image
    destination_input = "{}/page_{}.png".format(final_directory,page.number)
    pix.save(destination_input)
    return destination_input

def render_image_bw(destination_input):
    """ render image black and white to improve
    text extraction"""
    inputImage = cv2.imread(destination_input)

    # Conversion to CMYK (just the K channel):

    # Convert to float and divide by 255:
    imgFloat = inputImage.astype(np.float64) / 255.

    # Calculate channel K:
    kChannel = 1 - np.max(imgFloat, axis=2)

    # Convert back to uint 8:
    kChannel = (255 * kChannel).astype(np.uint8)
    
    destination_output = "{}/page_{}_out.png".format(final_directory,page.number)
    # Threshold image:
    binaryThresh = 190
    _, binaryImage = cv2.threshold(kChannel, binaryThresh, 255, cv2.THRESH_BINARY)
    #cv2.imshow("binaryImage [closed]", binaryImage)
    cv2.imwrite(destination_output, binaryImage)
    return destination_output

In [11]:
d_trascript = []

# loop through all files
for a in tqdm.tqdm(df['family_name'], total=df.shape[0]):
    try:
        report_str = '' # initialize empty text
        #print(a)
        file_path = os.path.join(folder_dichiarazione, a +'.pdf')  #pointer to file
        doc = fitz.open(file_path)  # open document

        final_directory = os.path.join(current_directory, folder_dichiarazione, a)  # store single pages documents in here
        if not os.path.exists(final_directory):
           os.makedirs(final_directory)
        for page in doc:
            destination_input = split_doc_png(page)  #png single pages
            destination_output = render_image_bw(destination_input)  #render image in black and white
            #print(destination_output)
            img = Image.open(destination_output)
            text = pytesseract.image_to_string(img, lang='ita')  #get text out of images
            report_str += text  #add all the pages

        # store results
        d_trascript.append(
            {
                'family_name': a,
                'transcript': report_str
            }
        )
    except:
        print('failed' + a)


  6%|██▍                                     | 19/315 [04:48<1:03:54, 12.95s/it]

failedBarboni


  8%|███▎                                      | 25/315 [05:55<58:59, 12.20s/it]

failedBertacco


  9%|███▊                                      | 29/315 [06:37<54:50, 11.51s/it]

...

 90%|███████████████████████████████████▏   | 284/315 [1:01:25<17:40, 34.22s/it]

failedStancanelli


 92%|████████████████████████████████████   | 291/315 [1:04:02<12:07, 30.30s/it]

failedTesei


100%|███████████████████████████████████████| 315/315 [1:09:53<00:00, 13.31s/it]


In [12]:
df_trascript = pd.DataFrame(d_trascript)
df_trascript.to_csv('df_trascript.csv', index=False)
df_trascript.head()

,family_name,transcript
0,Abate,Senato cella Repubblica\nServizio delle preros...
1,Accoto,. MopuLo C\nSénaio della Repubblica od\n\nServ...
2,Agostinelli,iS N n .\n2Ò Moputo C\nSenato della Repubblica...
3,Aimi,Senato delia Repubblica\n\nServi\nStoll\n\nell...
4,Airola,) MopuLo C\nSenato della Repubblica\n\nServizi...


In [13]:
df = pd.read_csv('df_trascript.csv')
df.head()

,family_name,transcript
0,Abate,Senato cella Repubblica\nServizio delle preros...
1,Accoto,. MopuLo C\nSénaio della Repubblica od\n\nServ...
2,Agostinelli,iS N n .\n2Ò Moputo C\nSenato della Repubblica...
3,Aimi,Senato delia Repubblica\n\nServi\nStoll\n\nell...
4,Airola,) MopuLo C\nSenato della Repubblica\n\nServizi...
